In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 29.0 MB/s 
     |████████████████████████████████| 6.5 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import os 
import sys
import pandas as pd
import numpy as np

import re

from transformers import AutoTokenizer, AutoModelForCausalLM 

In [ ]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
csv_test = pd.read_csv("/content/drive/MyDrive/NLP/sports_news_data - sports_news_data.csv")

In [ ]:
csv_test.head(20)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",[스포탈코리아] FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다. 잉여 ...,2021-07-15
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",[스포탈코리아=파리(프랑스)] 황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 ...,2021-07-15
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",[스포탈코리아] 리오넬 메시(34)가 FC바르셀로나에 남는다. 연봉을 절반 삭감하지...,2021-07-15
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",[스포탈코리아] 네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다. \n\...,2021-07-15
9,"100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",[스포탈코리아] 리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다. 전산화 하...,2021-07-15


In [ ]:
trimmed_data=csv_test.dropna().copy()
for i in range(len(trimmed_data)):
  if type(trimmed_data["CONTENT"].iloc[i]) != type("문자열"):
    continue

  trimmed_data["CONTENT"].iloc[i] = " ".join(trimmed_data["CONTENT"].iloc[i].split())
  rmve_bracket = "\[.*\]|\(.*\)|\<.+\>"
  trimmed_data["CONTENT"].iloc[i] = re.sub(rmve_bracket,"",trimmed_data["CONTENT"].iloc[i])
  mkd_mark = "[가-힣]{1,4}\s기자\=\s|사진=[가-힣]+"
  trimmed_data["CONTENT"].iloc[i] = re.sub(mkd_mark,"",trimmed_data["CONTENT"].iloc[i])

In [ ]:
drop_list = []
for i in range(len(trimmed_data)):
  if len(trimmed_data["CONTENT"].iloc[i]) < 20 :
    j = trimmed_data.iloc[i].name
    drop_list.append(j)
  else:
    trimmed_data["CONTENT"].iloc[i] = re.sub(r"[^\w\d가-힣\s+]","",trimmed_data["CONTENT"].iloc[i]).strip()
    trimmed_data["TITLE"].iloc[i] = re.sub(r"[^a-zA-Z0-9가-힣\s+]","",trimmed_data["TITLE"].iloc[i]).strip()

trimmed_data = trimmed_data.drop(index = drop_list)

In [ ]:
len(trimmed_data)

7259

In [ ]:
trimmed_data.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹를 주목했다 수비수가 볼을 터치하지 않았다고 봤다 멜레의 오른쪽 다리와 ...,2021-07-15
1,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까 지난달...,2021-07-15
2,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다 영국 매체 스포츠 바이...,2021-07-15
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다 이재성은 20...,2021-07-15
4,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다 팬들은 만족스럽다는 반응이다 바르...,2021-07-15


# Model-Load
- 네이버의 하이퍼클로바
- 기반은 GPT-3
- 기본적으로 생성에 강점이 있는 모델이라 선정함

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

In [ ]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 수수께끼들을 풀 수 있게 되었다. 지능은 지금까지 인간의 두뇌 속에서만 존재하는 것으로 여겨져 왔지만, 최근에는 인간의 몸속에서도 발견되었


#abstractive summarization